In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score


def data_select(fn_weather,fn_price):
    r_data = pd.read_csv(fn_weather, encoding="utf-8")
    data_t_n = ['Temperature', 'RH', 'StnPres', 'WS', 'WSGust','d_tmp']
    col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
    
    # rm "/"
    for col in col_fix:
        old = r_data[col]
        new = []
        for i in range(len(old)):
            try:
                new.append(float(old[i]))
                tmp = float(old[i])
            except ValueError:
                new.append(tmp)
        r_data[col] = new
    rain  = r_data['Precp']
    
    # rain_class function
    def rain_c(precp):
        rain_class=[0]
        rain_tag = ["no_rain", "rain"]
        result = 0
        for d in rain_class:
            if precp > d:
                result = result + 1
        return result

    rain_class=[]
    for precp in rain:
        rain_class.append(rain_c(precp))
    
    d_tmp = r_data['T.Max'] - r_data['T.Min']
    r_data["d_tmp"]= d_tmp
    d_data = r_data["Date"]
    drop_c =["Date",'T.Max', 'T.Min', 'Precp']
    r_data = r_data.drop(columns=drop_c)
    r_data["rain"]= rain_class
#     r_data.describe()

    def to_zscore(df, col_x):
    #     df_tmp = df
    #     col = "Temperature"
        for col in col_x:
            df[col] = stats.zscore(df[col])
            df[col] = np.round(df[col], decimals=4)
    # to_zscore(r_data, data_t_n)

    def to_zscore2(df):
        col_x = df.columns.to_list()
        mean = df.mean(axis=0)
        std = df.std(axis=0)
        for i in range(len(col_x)):
            df[col_x[i]]=(df[col_x[i]]-mean[i])/std[i]
        return  df , (mean, std)
    r_data, recordz = to_zscore2(r_data)
    
    return r_data, recordz , d_data

def data_shift(r_data, d_data, shift1, shift2=0):
    # shift and conbinde Data~ D-28 (D0~D-28)
    dfs =[]
    
    shifts = shift1
    
    if shift2==0:
        df_s = r_data.copy()
    else:
        df_s = r_data.copy()
        df_s = df_s.shift(periods=shift2)
        d_data = d_data[:(-shift2)]
        
#     add shift base
    arr_all = np.array(df_s)
    
    for i in range(1,shifts+1):
        tp = np.array(df_s.shift(periods=i))
        arr_all = np.concatenate((arr_all, tp), axis=1)
    
    df_all = pd.DataFrame(arr_all)
    
    df_all = df_all.dropna()
    df_all = df_all.reset_index()
    df_all = df_all.drop(columns="index")
    d_data = d_data[:(-shifts)]
    
    r_data = df_all

    d_date = d_data[0]
    
    def trans_to_y_w(d_date):
        year = int(d_date.split("-")[0])
        d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
        d_w = 1+ (d_day.days // 7)
        if d_w ==53:
            d_w = 52
        result =  str(year) + "_" + str(d_w)

        if d_day.days % 7 ==6:
            return result
        else:
            return None
    
    d_new = []
    for i in range(len(d_data)):
        d_new.append(trans_to_y_w(d_data[i]))
    r_data["y_w"] = d_new
    r_data = r_data.dropna()
    
    df_pbw = pd.read_csv(fn_price, encoding="utf-8")
    
    drop_c = ["year", "week"]
    df_pbw = df_pbw.drop(columns=drop_c)

    df_join = df_pbw.join(r_data.set_index("y_w"), on="y_w")
    df_join = df_join.dropna()
    
    d_t_dummy = ["market"]
    date_info = pd.DataFrame()
    for col in d_t_dummy:
        date_info[col] = df_join[col]
        dummy = pd.get_dummies(date_info[col])
        df_join = pd.concat([df_join, dummy], axis=1)
    
    drop_c = ["market", "w_avg", "w_sale", "y_w","price_diff"]
    y_raw = np.array(df_join["price_diff"])
#     print("y_price_diff describe:")
#     print(df_join["price_diff"].describe())
#     print("======")
    df_join = df_join.drop(columns=drop_c)
    
    def y_to_class(v):
        t = 0
        y_class_range = []
        for q in range(1,12):
            y_class_range.append(round((q*0.1-0.6),4))

        for i in range(len(y_class_range)):
            if (v >= y_class_range[i]):
                t = i+1
        return int(t)

    y_class = []

    for i in range(len(y_raw)):
        y_class.append(y_to_class(y_raw[i]))
#     print("freq:",np.bincount(y_class))
    
    
    
    x = np.array(df_join)
    y = y_class
    input_d = x.shape[1]
    
    
    return x, y, input_d

def model_run(x, y, input_d, shifts, epochs=40):
    
    out_class_count = len(np.bincount(y))
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    model = Sequential()
    layer_0 = Dense(units =input_d//4, input_dim = input_d,
                    kernel_initializer = "random_normal", activation = "relu")
    model.add(layer_0)
    model.add(Dropout(0.25))
    layer_1 = Dense(units =shifts,kernel_initializer = "random_normal", activation = "relu")
    model.add(layer_1)
    model.add(Dropout(0.25))
    layer_out = Dense(units = out_class_count,kernel_initializer = "random_normal", 
                      activation = "softmax")
    model.add(layer_out)
#     model.summary()
    model.compile(loss="categorical_crossentropy",optimizer = "adam", metrics = ['accuracy'])
    train_history = model.fit(x = x_train, y = np_utils.to_categorical(y_train), 
                              validation_split = 0.1, epochs =epochs, verbose = 0)
    
#     plt.plot(train_history.history["loss"])
#     plt.plot(train_history.history["val_loss"])
#     plt.title("Loss Graph")
#     plt.legend(['loss', 'val_loss'], loc="upper left")
    pre = model.predict_classes(x_test)
    acc = round(accuracy_score(y_test, pre)*100,2)
#     print("acc:", round(accuracy_score(y_test, pre)*100,2),"%")

    return acc, pd.DataFrame(confusion_matrix(y_test, pre)), model
    

    

Using TensorFlow backend.


In [2]:
# w_list= glob.glob("../result/merge_weather/*.csv")
# pd.set_option('display.max_rows', 20)
# w_list
# fn_weather = w_list[0]
# fn_price = "../result/flower_price_byweek/Rose_pbyweek.csv"


# shift1 = 10

# def test_base(fn_weather, fn_price, shift1):
#     r_data, recordz, d_data = data_select(fn_weather,fn_price)
#     plx = []
#     ply = []
#     for shft2 in range(7,42+1,7):
#         x, y, input_d = data_shift(r_data, d_data, shift1, shift2=0)
#         acc, df_conf, model = model_run(x, y, input_d, shift1, epochs=40)
#         print(shft2,acc)
#         plx.append(shft2)
#         ply.append(acc)

#     plt.plot(plx, ply)




# # 玫瑰 : 台中，彰化，雲林，嘉義，南投，高雄，屏東
# 洋桔梗 : 南投，彰化，雲林，嘉義，台南
# 大菊 : 彰化，雲林，南投
# 火鶴花 : 台中，嘉義，南投，台南，高雄，屏東
# 香水百合 :台中，南投，嘉義

# flowers = []
# for fn in get_f_list:
#     flowers.append(fn.split("\\")[-1].split("_")[0])

# flower = flowers[0]
# fn_list = glob.glob("../result/merge_flower_price/*" + flower + "*.csv")




In [3]:

# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# pd.set_option('display.max_rows', 20)
# # w_list
# # fn_weather = w_list[0]
# # fn_price = p_list[0]
# def test__shift1_allstations(w_list, p_list, shiftt2=0):
#     result_dataset =[]

#     for fn_weather in w_list:
#         print(fn_weather)
#         for fn_price in p_list:
#             print(fn_price)
#             rs_data = []
#             test_w = fn_weather.split("\\")[-1].split(".")[0] 
#             test_p = fn_price.split("\\")[-1].split("_")[0]
#             rs_data.append(test_w)
#             rs_data.append(test_p)
#             r_data, recordz, d_data = data_select(fn_weather,fn_price)
#             tmp_acc =[]
#             for shift1 in range(7,20,3):
#                 x, y, input_d = data_shift(r_data, d_data, shift1, shift2=7)
#                 acc, df_conf, model = model_run(x, y, input_d, shift1, epochs=40)
#                 tmp_acc.append(acc)
#             tmp_acc = np.array(tmp_acc)
#     #         rs_data.append(tmp_acc.mean())
#             rs_data.append(tmp_acc.max())
#             rs_data.append(tmp_acc.min())
#             result_dataset.append(rs_data)

#     df = pd.DataFrame(result_dataset,columns=["station_flower", "mean", "max", "min"])
#     df.to_csv("test_0517_shift2_7.csv", encoding="utf-8")
        
# # shift1 = 10
# # r_data, recordz, d_data = data_select(fn_weather,fn_price)
# # plx = []
# # ply = []
# # for shft2 in range(7,28+1,7):
# #     x, y, input_d = data_shift(r_data, d_data, shift1, shift2=0)
# #     acc, result = model_run(x, y, input_d, shift1, epochs=40)
# #     print(shft2,acc)
# #     plx.append(shft2)
# #     ply.append(acc)

# # plt.plot(plx, ply)

In [ ]:
w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/flower_price_byweek/*")
select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]


def run_model_build(fn_weather,fn_price,shift2=0):
    rs_data = []
    test_w = fn_weather.split("\\")[-1].split(".")[0] 
    test_p = fn_price.split("\\")[-1].split("_")[0]
    rs_data.append(test_w)
    rs_data.append(test_p)
    print(rs_data)
    r_data, recordz, d_data = data_select(fn_weather,fn_price)
    rs_data.append(recordz)
    tmp_acc =[]
    acc_cf = 0
    for shift1 in range(5,14+1,2):
        print("shift=", shift1)
        for epochs in range(30,80+1,10):
            x, y, input_d = data_shift(r_data, d_data, shift1, shift2)
            acc, df_conf, model = model_run(x, y, input_d, shift1, epochs)
            tmp_acc.append(acc)
            if acc > acc_cf:
                acc_cf = acc
                best_model = model
                set_info = (shift1, shift2, epochs)
                print("epochs = ", epochs,"acc = ",acc)
    rs_data.append([acc_cf, set_info])        
    rs_data.append(best_model)
    return rs_data


# for st, flower in select_t:
st, flower = select_t[0]
fn_weather = w_list[st]
fn_price = p_list[flower]
result = run_model_build(fn_weather,fn_price)

result

['C0F9L0_后里', 'Anthurium']
shift= 5
epochs =  30 acc =  37.52
epochs =  60 acc =  39.65
epochs =  80 acc =  40.28
shift= 7
epochs =  30 acc =  43.54
epochs =  40 acc =  49.06


In [ ]:
# w_list= glob.glob("../result/merge_weather/*.csv")
# p_list = glob.glob("../result/flower_price_byweek/*")
# pd.set_option('display.max_rows', 20)
# w_list
# wc = [0,1,4,-1]
# w_list = list(np.array(w_list)[wc])

# result_dataset =[]
# def run_model_build(w_list,p_list):
#     rs_data = []
#     test_w = fn_weather.split("\\")[-1].split(".")[0] 
#     test_p = fn_price.split("\\")[-1].split("_")[0]
#     rs_data.append(test_w)
#     rs_data.append(test_p)
#     r_data, recordz, d_data = data_select(fn_weather,fn_price)
#     tmp_acc =[]
#     for shift1 in range(7,20,4):
#         x, y, input_d = data_shift(r_data, d_data, shift1, shift2=14)
#         acc, df_conf, model = model_run(x, y, input_d, shift1, epochs=40)
#         tmp_acc.append(acc)
#     tmp_acc = np.array(tmp_acc)
# #         rs_data.append(tmp_acc.mean())
#     rs_data.append(tmp_acc.max())
#     rs_data.append(tmp_acc.min())
#     result_dataset.append(rs_data)
#     return result_dataset


# df = pd.DataFrame(result_dataset,columns=["station", "flower", "max", "min"])
# df.to_csv("test_0517_shift2_14.csv", encoding="utf-8")